In [1]:
import os
import sys

def safe_pip_install(package):
    try:
        # Detect Colab
        in_colab = 'google.colab' in sys.modules

        # Use shell-style install in Colab
        if in_colab:
            print(f"Installing {package} in Colab...")
            # Use !pip or %pip to avoid subprocess errors
            get_ipython().system(f"pip install {package}")
        else:
            print(f"Installing {package} in standard environment...")
            # Use subprocess for non-Colab environments
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except Exception as e:
        print(f"Failed to install {package}: {e}")

In [2]:
import os
import torch

def detect_environment():
    # Check for GPU
    has_gpu = torch.cuda.is_available()

    # Check for Colab-specific environment
    is_colab = 'COLAB_GPU' in os.environ or 'google.colab' in str(get_ipython())

    # Check for RunPod-specific environment
    is_runpod = 'RUNPOD_POD_ID' in os.environ or os.path.exists('/workspace')

    if is_runpod and has_gpu:
        return 'runpod'
    elif is_colab and not has_gpu:
        return 'colab'
    else:
        return 'unknown'

#check the environement
env = detect_environment()

if env == 'runpod':
    print("Running on RunPod with GPU — using full model.")
    # Load full CheXagent, enable CUDA, etc.
    #GPU update:
    safe_pip_install("transformers==4.40.0")
    safe_pip_install("torch")
    safe_pip_install("torchvision")
    safe_pip_install("pydicom")
    safe_pip_install("opencv-python")
    safe_pip_install("Pillow")
    safe_pip_install("accelerate")
elif env == 'colab':
    print("Running on Colab with CPU — using lightweight fallback.")
    safe_pip_install("transformers==4.40.0")
    safe_pip_install("transformers==4.40.0")
    safe_pip_install("torch")
    safe_pip_install("torchvision")
    safe_pip_install("pydicom")
    safe_pip_install("opencv-python")
    safe_pip_install("Pillow")
    safe_pip_install("accelerate")
else:
    print("Unknown environment — defaulting to safe config.")


Running on Colab with CPU — using lightweight fallback.
Installing transformers==4.40.0 in Colab...
Installing transformers==4.40.0 in Colab...
Installing torch in Colab...
Installing torchvision in Colab...
Installing pydicom in Colab...
Installing opencv-python in Colab...
Installing Pillow in Colab...
Installing accelerate in Colab...


In [3]:
# Install dependencies
#Imports
import os, glob
import torch
import pydicom
import cv2
from PIL import Image
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [42]:
from pathlib import Path

#check the environement
env = detect_environment()
if env == 'runpod':
  ##rclone sync gdrive:/MyDrive/MLProjects/foundation-models-radiology /workspace/MLProjects/foundation-models-radiology
  ROOT = Path('/workspace/MLProjects/foundation-models-radiology')
elif env == 'colab':
  from google.colab import drive
  drive.mount('/content/drive')
  ###once mounted the folders can be referenced
  ROOT = Path('/content/drive/MyDrive/MLProjects/foundation-models-radiology')

else:
  sys.exit("Error: No platform recognised")

DICOM_DIR = ROOT / 'PTXHeadtoHeadSmall'   # use the exact folder name as on Drive
JPEG_DIR = ROOT / 'cxr_jpegs'
JPEG_DIR.mkdir(exist_ok=True)
print("exists:", ROOT.exists())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
exists: True


In [16]:
#Set paths

#ROOT = Path("/content/drive/MyDrive")  # adjust if needed
#DICOM_DIR = ROOT / "PTXHeadtoHeadSmall"

#Replace with: direct file access or use rclone to sync your Google Drive into the pod’s local filesystem.
#RunPod doesn’t support drive.mount().
#GPU update:
#ROOT = Path('/workspace/MLProjects/foundation-models-radiology')
#JPEG_DIR = ROOT / 'cxr_jpegs'

exists: True


In [43]:
# Convert DICOMs to JPEGs
def dicom_to_jpeg(dicom_path, jpeg_path):
    ds = pydicom.dcmread(str(dicom_path))
    img = ds.pixel_array
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    img = cv2.equalizeHist(img)  # optional contrast enhancement
    cv2.imwrite(str(jpeg_path), img)

# Batch convert
#dicom_paths = list(DICOM_DIR.rglob("*.dcm"))
# find DICOMs (case-insensitive .dcm)
dicom_paths = list(DICOM_DIR.rglob('*.[dD][cC][mM]'))
print("count:", len(dicom_paths))
print("samples:", dicom_paths[:5])

jpeg_paths = []
for dcm_path in dicom_paths:
    jpg_path = JPEG_DIR / f"{dcm_path.stem}.jpg"
    dicom_to_jpeg(dcm_path, jpg_path)
    jpeg_paths.append(str(jpg_path))


count: 2
samples: [PosixPath('/content/drive/MyDrive/MLProjects/foundation-models-radiology/PTXHeadtoHeadSmall/0a1d53cc79cb2ee50afa507d425f6d15/04797ae93ba9e94174bd928999395887/204611bfbd3e2c990bc2e9d6e131d46a/813f8b5f7031f381d6aadb2edcdd15cf.dcm'), PosixPath('/content/drive/MyDrive/MLProjects/foundation-models-radiology/PTXHeadtoHeadSmall/ffac9214400053aa4c78126255328235/113cd0ccb7393ecb0a5a8a94dac38f72/73cedef0efa5a294616db1be804bd026/3dc870d5ffa18cb81c81342ca478eb4f.dcm')]


In [4]:
### step 2: Load Processor and Model from HuggingFace
##tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
##model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
##model = model.to(dtype)
##model.eval()
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "StanfordAIMI/CheXagent-2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    low_cpu_mem_usage=True
)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json: 0.00B [00:00, ?B/s]

configuration_chexagent.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/CheXagent-2-3b:
- configuration_chexagent.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_chexagent.py: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
####format data for fine-tuning
{
  "image": PIL.Image,
  "instruction": "What abnormalities are present?",
  "answer": "There is bilateral interstitial infiltrate consistent with pulmonary edema."
}


In [39]:
for name, param in model.named_parameters():
    if param.device.type == "meta":
        print(f"{name} is on the meta device")

model.layers.28.self_attn.q_proj.weight is on the meta device
model.layers.28.self_attn.q_proj.bias is on the meta device
model.layers.28.self_attn.k_proj.weight is on the meta device
model.layers.28.self_attn.k_proj.bias is on the meta device
model.layers.28.self_attn.v_proj.weight is on the meta device
model.layers.28.self_attn.v_proj.bias is on the meta device
model.layers.28.self_attn.dense.weight is on the meta device
model.layers.28.self_attn.dense.bias is on the meta device
model.layers.28.mlp.fc1.weight is on the meta device
model.layers.28.mlp.fc1.bias is on the meta device
model.layers.28.mlp.fc2.weight is on the meta device
model.layers.28.mlp.fc2.bias is on the meta device
model.layers.28.input_layernorm.weight is on the meta device
model.layers.28.input_layernorm.bias is on the meta device
model.layers.29.self_attn.q_proj.weight is on the meta device
model.layers.29.self_attn.q_proj.bias is on the meta device
model.layers.29.self_attn.k_proj.weight is on the meta device
mo

In [74]:
# step 3: Inference
from PIL import Image

def ask_chexagent(image_path, question):
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        #basically the same **query = tokenizer.from_list_format([*[{'image': path} for path in paths], {'text': prompt}])
        query = tokenizer.from_list_format([
            {"image": str(image_path)},
            {"text": question}
        ])
        conversation = [
            {"from": "system", "value": "You are a helpful assistant."},
            {"from": "human",  "value": query}
        ]
        # Returns a tensor, not a dict
        #input_ids = tokenizer.apply_chat_template(conv, add_generation_prompt=True, return_tensors="pt") **HF
        input_ids = tokenizer.apply_chat_template(
            conversation, add_generation_prompt=True, return_tensors="pt"
        ).to(model.device)
        # Attention mask (optional; model can infer it, but this is safe)
        attention_mask = input_ids.ne(tokenizer.pad_token_id) if tokenizer.pad_token_id is not None else None

        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,  # ← this line
            do_sample=False,
            num_beams=1,
            temperature=1.0,
            top_p=1.0,
            use_cache=True,
            max_new_tokens=512
        )[0]

        response = tokenizer.decode(output, skip_special_tokens=True)
        return response.strip()

    except Exception as e:
        return f" Error processing {image_path}: {e}"


In [75]:
# Run on all images and collect responses
import csv
# Define query
question = "Does this chest X-ray show a pneumothorax?"

results = []
for path in jpeg_paths:
    answer = ask_chexagent(path, question)
    results.append((path, answer))
    print(f" {Path(path).name} → {answer}")

with open("chexagent_results.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Answer"])
    writer.writerows(results)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 813f8b5f7031f381d6aadb2edcdd15cf.jpg →  Error processing /content/drive/MyDrive/MLProjects/foundation-models-radiology/cxr_jpegs/813f8b5f7031f381d6aadb2edcdd15cf.jpg: Tensor on device meta is not on the expected device cpu!
 3dc870d5ffa18cb81c81342ca478eb4f.jpg →  Error processing /content/drive/MyDrive/MLProjects/foundation-models-radiology/cxr_jpegs/3dc870d5ffa18cb81c81342ca478eb4f.jpg: Tensor on device meta is not on the expected device cpu!
